In [121]:
import bs4 as bs
import urllib.request
import pandas as pd
import re
import datetime
import pymysql

### 1. Data Scraping 

#### url 입력 (자료가 있는 정례브리핑 url 주소 입력)

In [122]:
url = urllib.request.urlopen(input()).read()
soup = bs.BeautifulSoup(url, 'lxml')

http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=354184&contSeq=354184&board_id=&gubun=ALL


#### pandas를 이용한 table scraping

In [123]:
dfs = pd.read_html(url, encoding = "utf-8", header=0)

In [124]:
dfs[0]

,구분,구분.1,총계,결과 양성,결과 양성.1,결과 양성.2,결과 양성.3,검사 중,결과 음성
0,구분,구분,총계,확진자,격리해제,격리 중,사망,검사 중,결과 음성
1,4. 22.(수) 0시 기준,4. 22.(수) 0시 기준,577959,10694,8277,2179,238,12121,555144
2,4. 23.(목) 0시 기준,4. 23.(목) 0시 기준,583971,10702,8411,2051,240,10139,563130
3,변동,변동,"(+)6,012",(+)8,(+)134,(-)128,(+)2,"(-)1,982","(+)7,986"


#### domestic_check table에 들어갈 내용 scraping

In [126]:
column = dfs[0].columns
date = dfs[0][column[0]][2]
regex = re.compile(r'\d.')
date_group = regex.findall(date)

for i in range(0, len(date_group)):
    date_group[i] = int(re.sub("[^\d]", "", date_group[i]))
    
report_day = datetime.datetime(2020,date_group[0],date_group[1])
report_day_str = report_day.strftime('%Y-%m-%d')

check = []

for i in range(2, 9):
    check.append(int(dfs[0][column[i]][2]))
check.append(int(dfs[0][column[3]][3].replace("(+)", "")\
                                          .replace("(-)", "-")\
                                          .replace(",", "")))
check.append(int(dfs[0][column[4]][3].replace("(+)", "")\
                                          .replace("(-)", "-")\
                                          .replace(",", "")))
check.insert(0, report_day_str)
check

['2020-04-23', 583971, 10702, 8411, 2051, 240, 10139, 563130, 8, 134]

#### domestic_sid table에 들어갈 내용 scraping

In [127]:
column = dfs[1].columns
sid = []

for i in range(0, 3):
    list_temp = []
    for j in range(1, len(column)):
        list_temp.append(int(dfs[1][column[j]][i]))
    if i == 0 : list_temp.insert(0, "under quarantine")
    elif i == 1 : list_temp.insert(0, "end quarantine")
    elif i == 2 : list_temp.insert(0, "death")
    list_temp.insert(0, report_day_str)
    sid.append(list_temp)
sid

[['2020-04-23',
  'under quarantine',
  2051,
  284,
  16,
  784,
  36,
  6,
  11,
  6,
  12,
  221,
  20,
  5,
  18,
  8,
  7,
  220,
  28,
  5,
  364],
 ['2020-04-23',
  'end quarantine',
  8411,
  342,
  116,
  5890,
  56,
  24,
  29,
  36,
  34,
  424,
  31,
  40,
  123,
  9,
  8,
  1091,
  89,
  8,
  61],
 ['2020-04-23',
  'death',
  240,
  2,
  3,
  166,
  0,
  0,
  0,
  1,
  0,
  14,
  2,
  0,
  0,
  0,
  0,
  52,
  0,
  0,
  0]]

#### domestic_group table에 들어갈 내용 scraping

In [136]:
location = ["Seoul", "Busan", "Daegu", "Incheon",
 "Gwangju", "Daejeon", "Ulsan", "Sejong",
 "Gyeonggi", "Gangwon", "Chungbuk", "Chungnam",
 "Jeonbuk", "Jeonnam", "Gyeongbuk", "Gyeongnam", "Jeju", "Quarantine"]
column = dfs[2].columns
group = []

for i in range(2, 20):
    list_temp = []
    for j in range(1, 12):
        list_temp.append(int(dfs[2][column[j]][i]))
    list_temp.insert(0, location[i-2])
    list_temp.insert(0, report_day_str)
    list_temp.pop(5)
    list_temp.pop(10)
    group.append(list_temp)
group

[['2020-04-23', 'Seoul', 628, 247, 357, 8, 262, 18, 69, 24, 0],
 ['2020-04-23', 'Busan', 135, 22, 84, 12, 53, 18, 1, 29, 1],
 ['2020-04-23', 'Daegu', 6840, 18, 6100, 4510, 662, 924, 4, 722, 4],
 ['2020-04-23', 'Incheon', 92, 41, 47, 2, 34, 5, 6, 4, 0],
 ['2020-04-23', 'Gwangju', 30, 14, 16, 9, 0, 1, 6, 0, 0],
 ['2020-04-23', 'Daejeon', 40, 10, 21, 2, 11, 8, 0, 9, 0],
 ['2020-04-23', 'Ulsan', 43, 12, 24, 16, 1, 4, 3, 7, 0],
 ['2020-04-23', 'Sejong', 46, 3, 42, 1, 38, 3, 0, 1, 0],
 ['2020-04-23', 'Gyeonggi', 659, 152, 466, 29, 320, 55, 62, 41, 1],
 ['2020-04-23', 'Gangwon', 53, 14, 32, 17, 15, 0, 0, 7, 0],
 ['2020-04-23', 'Chungbuk', 45, 5, 32, 6, 18, 6, 2, 8, 0],
 ['2020-04-23', 'Chungnam', 141, 11, 120, 0, 118, 1, 1, 10, 0],
 ['2020-04-23', 'Jeonbuk', 17, 8, 4, 1, 3, 0, 0, 5, 0],
 ['2020-04-23', 'Jeonnam', 15, 7, 7, 1, 3, 2, 1, 1, 0],
 ['2020-04-23', 'Gyeongbuk', 1363, 10, 1219, 566, 461, 192, 0, 134, 0],
 ['2020-04-23', 'Gyeongnam', 117, 14, 85, 32, 46, 6, 1, 18, 0],
 ['2020-04-23', '

### 2. PYTHON과 MYSQL 연동

In [137]:
conn = pymysql.connect(host = '192.168.0.2', user = 'vanillapapaya',
                      password = 'qhans7810!', db = 'covid',
                      charset = 'utf8')
curs = conn.cursor()

#### domestic_check에 column 추가

In [138]:
query = """

        INSERT INTO domestic_check(Date, Total, Positive, EndQuarantine, UnderQuarantine, Death,
                               UnderExamination, Negative, NewPositive, NewEndQuarantine)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        
"""

curs.execute(query, check)

1

#### domestic_sid에 column 추가

In [139]:
query = """

        INSERT INTO domestic_sid(Date, Type, Total, Seoul, Busan, Daegu, Incheon, Gwangju, Daejeon,
                               Ulsan, Sejong, Gyeonggi, Gangwon, Chungbuk, Chungnam, Jeonbuk, Jeonnam,
                               Gyeongbuk, Gyeongnam, Jeju, Quarantine)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)

"""
curs.executemany(query, sid)

3

#### domestic_group에 column 추가

In [140]:
query = """

        INSERT INTO domestic_group(Date, Location, Total, ForeignInflow, GroupTotal, 
                                    Shincheonji, GroupCases, PersonsInContact, ForeignGroupInflow, Other, New)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)

"""
curs.executemany(query, group)

18

#### 변경된 내용 확인

In [143]:
query = """

    SELECT *
        FROM domestic_check

"""

df = pd.read_sql(query, conn)
df

,index,Date,Positive,EndQuarantine,UnderQuarantine,Death,UnderExamination,Negative,Total,NewPositive,NewEndQuarantine
0,0.0,2020-01-21,1,0,1,0,3,7,11,0,0
1,1.0,2020-01-24,2,0,2,0,0,25,27,1,0
2,2.0,2020-01-26,3,0,3,0,1,47,51,1,0
3,3.0,2020-01-27,4,0,4,0,1,56,61,1,0
4,4.0,2020-01-30,6,0,6,0,41,199,246,2,0
...,...,...,...,...,...,...,...,...,...,...,...
83,83.0,2020-04-19,10661,8042,2385,234,12243,536205,559109,8,105
84,84.0,2020-04-20,10674,8114,2324,236,11981,540380,563035,13,72
85,NaN,2020-04-21,10683,8213,2233,237,12721,547610,571014,9,99
86,NaN,2020-04-22,10694,8277,2179,238,12121,555144,577959,11,64


#### QUERY 실행을 완료한 후 MYSQL에 최종 적용

In [144]:
conn.commit()